# Importing library

In [1]:
import os
import torch
from PIL import Image
import opendatasets as od
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Data Setting

In [2]:
batch_size = 4

## Monet dataset

In [3]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Assuming RGB images
])

In [4]:
monet_dataset_folderurl=r'gan-getting-started\monet_jpg'

In [5]:
monet_tensor = []


for file in os.listdir(monet_dataset_folderurl):
    image_path = os.path.join(monet_dataset_folderurl, file)
    image = Image.open(image_path)
    image = transform(image)
    monet_tensor.append(image)

In [6]:
Monet_dl=DataLoader(monet_tensor, batch_size, shuffle=True, num_workers=2, pin_memory=False )

## Photos Dataset

In [7]:
photo_dataset_folderurl=r'gan-getting-started\photo_jpg'

In [8]:
photos_tensor=[]

for file in os.listdir(photo_dataset_folderurl):
    image_path = os.path.join(photo_dataset_folderurl, file)
    image = Image.open(image_path)
    image = transform(image)  # Assuming transform is a torchvision transform
    photos_tensor.append(image)

In [9]:
photos_dl=DataLoader(photos_tensor, batch_size, shuffle=True, num_workers=2, pin_memory=False )

# Model

## Style transfer model

In [10]:
class StyleTransfer(nn.Module):
    def __init__(self):
        super(StyleTransfer, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # shape: (batch_size, 32, 128, 128)
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # shape: (batch_size, 64, 64, 64)
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), # shape: (batch_size, 128, 32, 32)
            nn.ReLU(inplace=True)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # shape: (batch_size, 64, 64, 64)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),   # shape: (batch_size, 32, 128, 128)
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1),    # shape: (batch_size, 3, 256, 256)
            nn.Sigmoid()  # Using Sigmoid to get the output in the range [0, 1]
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
model = StyleTransfer()

## Discriminator 

In [12]:
discriminator = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=2),  # shape: (batch_size, 32, 128, 128)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=2),  # shape: (batch_size, 64, 64, 64)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(64, 32, kernel_size=4, stride=2, padding=2),  # shape: (batch_size, 32, 32, 32)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(32, 16, kernel_size=4, stride=2, padding=2),  # shape: (batch_size, 16, 16, 16)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(16, 8, kernel_size=4, stride=2, padding=2),   # shape: (batch_size, 8, 8, 8)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(8, 4, kernel_size=4, stride=2, padding=2),    # shape: (batch_size, 4, 4, 4)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(4, 2, kernel_size=4, stride=2, padding=2),    # shape: (batch_size, 2, 2, 2)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(2, 1, kernel_size=4, stride=2, padding=2),    # shape: (batch_size, 1, 1, 1)
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(1, 1, kernel_size=2, stride=1, padding=0),    # shape: (batch_size, 1, 1, 1)
    nn.Flatten(),
    nn.Sigmoid()
)


In [13]:
x =discriminator(monet_tensor[0].unsqueeze(0))

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [15]:
model = model.to(device)
discriminator = discriminator.to(device)

In [16]:
k=0
def save_progress(k,model, save=False):
 

 image_np = photos_tensor[3].cpu().detach().numpy().transpose(1, 2, 0)
 image_np = torch.from_numpy(image_np)

 image_np = image_np.to(device)

 model_output_image= model(photos_tensor[3]).cpu().detach().numpy().transpose(1, 2, 0)

# Create a figure and a grid of subplots
 fig, axs = plt.subplots(1, 2, figsize=(10, 5))

 axs[0].imshow(image_np)
 axs[0].set_title('Original Image')
 axs[0].axis('off')

# Plot the model output image
 axs[1].imshow(model_output_image)
 axs[1].set_title('Model Output')
 axs[1].axis('off')

# Adjust the layout to make room for titles

 #plt.tight_layout()
 if save:
  fig.savefig(r'Output\output_images__{i}.png'.format(i=k))
  plt.close()
 return k


In [41]:

def save_progress(k, model,  device, save=False):
    # Select the 4th image from the photos_tensor and convert it to a numpy array
  
    image_np = photos_tensor[3].cpu().detach().numpy().transpose(1, 2, 0)
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    image_np = (image_np * std) + mean
    
    # Convert numpy array back to a tensor and move it to the specified device
    image_tensor = torch.from_numpy(image_np).to(device)

    # Get the model's output for the 4th image
    #model_output_image = model(photos_tensor[3].unsqueeze(0).to(device)).cpu().detach().numpy().transpose(0, 2, 3, 1).squeeze(0)
    model_output_image = model(photos_tensor[3].unsqueeze(0).to(device)).cpu().detach().numpy().transpose(0, 2, 3, 1).squeeze(0)
    
    #model_output_image = (model_output_image * std) + mean

    # Create a figure and a grid of subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the original image
    axs[0].imshow(image_np)
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    # Plot the model output image
    axs[1].imshow(model_output_image)
    axs[1].set_title('Model Output')
    axs[1].axis('off')

    # Adjust the layout to make room for titles
    plt.tight_layout()

    # Save the figure if needed
    if save:
        fig.savefig(r'Output\output_images_{i}.png'.format(i=k))
        output_path = os.path.join(r'output2', 'output_images_{i}.png'.format(i=k))
        plt.imsave(output_path, model_output_image)
        plt.close()
    
    return k + 1

In [52]:
def save_progress(k, model, device, save=False):
    # Create a fixed image tensor (assuming photos_tensor is a tensor containing images)
    fixed_image = photos_tensor[3]

    # Convert the fixed image tensor to a numpy array
    fixed_image_np = fixed_image.cpu().detach().numpy().transpose(1, 2, 0)
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    fixed_image_np = (fixed_image_np * std) + mean

    # Convert the numpy array back to a tensor and move it to the specified device
    fixed_image_tensor = torch.from_numpy(fixed_image_np).to(device)

    # Get the model's output for the fixed image
    model_output_image = model(fixed_image_tensor.unsqueeze(0).to(device)).cpu().detach().numpy().transpose(0, 2, 3, 1).squeeze(0)

    # Create a figure and a grid of subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Plot the fixed image
    axs[0].imshow(fixed_image_np)
    axs[0].set_title('Fixed Image')
    axs[0].axis('off')

    # Plot the model output image
    axs[1].imshow(model_output_image)
    axs[1].set_title('Model Output')
    axs[1].axis('off')

    # Adjust the layout to make room for titles
    plt.tight_layout()

    # Save the figure and the model output image if needed
    if save:
        output_dir = 'output2'
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, 'output_images_{i}.png'.format(i=k))
        plt.savefig(output_path)
        plt.imsave(output_path.replace('.png', '_model_output.png'), model_output_image)
        plt.close()

    return k + 1


In [53]:
k= save_progress(k+1,model, device, save= False )

RuntimeError: Input type (double) and bias type (float) should be the same

# Train discriminator

In [ ]:
import torch.nn.functional as F

In [ ]:
def train_discriminator(real_img,fake_img ,opt_d, discriminator):
    #clear discriminator grad
    opt_d.zero_grad()
    real_img = real_img.to(device)
    fake_img = fake_img.to(device)

    #passreal imageto discriminaor
    real_pred=discriminator(real_img)
   # real_target = torch.ones(real_img.size(0), 1)
    real_target = torch.ones(real_img.size(0), 1).to(device)
    real_loss = F.binary_cross_entropy(real_pred, real_target)
    real_score = torch.mean(real_pred).item()
   

    #pass fake  image through discriminator    #fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
    fake_pred = discriminator(fake_img)
    #fake_target = torch.zeros(fake_img.size(0), 1)
    fake_target = torch.zeros(fake_img.size(0), 1).to(device)
    fake_loss =  F.binary_cross_entropy(fake_pred, fake_target)
    fake_score = torch.mean(fake_pred).item()
    

    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()
    return loss.item(), real_score, fake_score



In [ ]:
criterion = nn.BCELoss()

In [ ]:
def train_generator(model, discriminator,opt_g, real_img):
    # Clear generator gradients
    opt_g.zero_grad()

    # Generate fake images
    fake_images = model(real_img)

    # Try to fool the discriminator
    preds = discriminator(fake_images)
    #targets = torch.zeros_like(preds)  # Use zeros as targets for fake images
    #targets = torch.zeros(preds.size(0), 1)
    targets = torch.zeros(batch_size, 1, device=device) #changed ones to zeros
    targets = targets.to(device)
    #loss = criterion(preds, torch.ones_like(preds))
    loss = F.binary_cross_entropy(preds, targets)

    # Update generator weights
    loss.backward()
    opt_g.step()

    return loss.item()


In [ ]:
from tqdm.notebook import tqdm

# Training model

In [ ]:
import os, re
def get_k():
 try:
  files = sorted(os.listdir(r'Output'))
  numbers = [int(re.search(r'\d+', filename).group()) for filename in files]
  k = max(numbers)
 except:
  k = 0
 return k

In [ ]:
def fit(epochs, lr):
    torch.cuda.empty_cache()
    k = get_k()

    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []

    # Create optimizers
    opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))
#    opt_g = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in tqdm(range(epochs)):
        #for real_images , fake_images in tqdm(zip(Monet_dl, photos_dl)):
        for real_images , fake_images in zip(Monet_dl, photos_dl):
            real_images = real_images.to(device)
            fake_images = fake_images.to(device)
            # Train discriminator
            loss_d, real_score, fake_score = train_discriminator(real_images, fake_images,opt_d, discriminator)
            #loss_d, real_score, fake_score = train_discriminator(real_images,opt_d)
            #if epoch % 100 == 0:
            loss_g  = train_generator(model, discriminator, opt_g, fake_images)
            
            
        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)

        # Log losses & scores (last batch)
        if epoch % 10 == 0:
         print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
           epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
         k= save_progress(k+1,model,device,  save= True )



        # Save generated images

    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
lr = 0.0002
epochs = 1000

In [ ]:
history = fit(epochs, lr)

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1/1000], loss_g: 0.5536, loss_d: 1.0466, real_score: 0.5846, fake_score: 0.3274


KeyboardInterrupt: 

In [ ]:
losses_g, losses_d, real_scores, fake_scores = history

In [ ]:
plt.plot(losses_d, '-')
plt.plot(losses_g, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real', 'Fake'])
plt.title('Scores');

# Save Model

In [ ]:
torch.save(model.state_dict(), 'model.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')